# [4차 프로그래밍 과제]

### 1. II. 데이터 셋에서 주어진 특징들을 선택하여 CPU 성능 즉, “PRP” 변수를 예측하는 Pipeline 기반 다중 회귀 모델링을 프로그래밍 하시오.
* 데이터 탐색 및 시각화, 전처리, 훈련 데이터와 테스트 데이터 분리, 하이퍼파라미터 튜닝, 훈련 데이터 앙상블 학습 과정, 교차 검증, 분석 결과 시각화 등을 포함하여 최적화하는 전체 과정을 포함하시오.

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV # 교차검증
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline # 파이프라인
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor # 앙상블

In [21]:
# 데이터 로드 
data = pd.read_csv("machine_data.csv")
print(data.columns)

# 범주형 변수 더미 변수화
data = pd.get_dummies(data, columns=['VendorName', 'ModelName'])

# 특성과 목표 변수 분리
X = data.drop(['PRP'], axis=1)  # PRP를 제외한 모든 열을 특성으로 사용
y = data['PRP']  # PRP 열을 목표 변수로 사용

# 전처리 파이프라인 설정
num_features = X.columns
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features)
])

# 개별 회귀 모델 정의
models = [
    ('lr', LinearRegression()),
    ('sgd', SGDRegressor()),
    ('ridge', Ridge())
]

# 앙상블 모델 정의
ensemble = VotingRegressor(estimators=models)

# 모델 파이프라인 생성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ensemble', ensemble)
])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 서치 설정
params_grid = {
    "ensemble__sgd__alpha": [0.0001, 0.1, 1.0],
    "ensemble__sgd__max_iter": [1000, 2000, 3000],
    "ensemble__ridge__alpha": [0.1, 1.0, 10.0]
}

# GridSearchCV 설정 및 모델 학습
grid_search = GridSearchCV(model_pipeline, params_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

Index(['VendorName', 'ModelName', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN',
       'CHMAX', 'PRP'],
      dtype='object')


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'VendorName_adviser',
       'VendorName_amdahl', 'VendorName_apollo', 'VendorName_basf',
       ...
       'ModelName_v8635', 'ModelName_v8650', 'ModelName_v8655',
       'ModelName_v8665',...
       'ModelName_vs-90'],
      dtype='object', length=245))])),
                                       ('ensemble',
                                        VotingRegressor(estimators=[('lr',
                                                                     LinearRegression()),
                                                                    ('sgd',
                                                                     SGDRegressor()),
                                                                    ('ridge',
                                                                     Ridge())]))]),
             param_grid={'ensemble__ridge__alpha': [0.1, 1.0, 10.0],
                         'ensemble__sgd__alpha': [0.0001, 0.1, 1.0],
                         'ensemble__sgd__max_iter': [1000, 2000, 3000]},
             return_train_score=True, scoring='neg_mean_squared_error')

### 2. (1) 단계에서 얻은 Pipeline 기반 모델 GridSearchCV의 best_estimator모델을 사용하여 학습 데이터와 테스트 데이터 각각에 대해 이 모델의 성능을 평가하시오.
* Mean squared Error, Mean absolute Error, R^2, 교차검증

In [31]:
# 데이터 로드 
data = pd.read_csv("machine_data.csv")
print(data.columns)

# 범주형 변수 더미 변수화
data = pd.get_dummies(data, columns=['VendorName', 'ModelName'])

# 특성과 목표 변수 분리
X = data.drop(['PRP'], axis=1)  # PRP를 제외한 모든 열을 특성으로 사용
y = data['PRP']  # PRP 열을 목표 변수로 사용

# 전처리 파이프라인 설정
num_features = X.columns
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features)
])

# 개별 회귀 모델 정의
models = [
    ('lr', LinearRegression()),
    ('sgd', SGDRegressor()),
    ('ridge', Ridge())
]

# 앙상블 모델 정의
ensemble = VotingRegressor(estimators=models)

# 모델 파이프라인 생성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ensemble', ensemble)
])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 서치 설정
params_grid = {
    "ensemble__sgd__alpha": [0.0001, 0.1, 1.0],
    "ensemble__sgd__max_iter": [1000, 2000, 3000],
    "ensemble__ridge__alpha": [0.1, 1.0, 10.0]
}

# GridSearchCV 설정 및 모델 학습
grid_search = GridSearchCV(model_pipeline, params_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

# 최적 모델 및 결과 출력
best_model = grid_search.best_estimator_

# 학습 데이터에 대한 성능 평가
y_train_pred = best_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# 교차 검증을 통한 성능 평가 (MSE, MAE, R^2)
cv_scores_mse = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores_mae = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
cv_scores_r2 = cross_val_score(best_model, X_train, y_train, cv=5, scoring='r2')

# 테스트 데이터에 대한 성능 평가
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# 결과 출력
print("Train MSE:", train_mse)
print("Train MAE:", train_mae)
print("Train R^2:", train_r2)
print("Test MSE:", test_mse)
print("Test MAE:", test_mae)
print("Test R^2:", test_r2)
print("CV MSE:", -cv_scores_mse)
print("CV MAE:", -cv_scores_mae)
print("CV R^2:", cv_scores_r2)


Index(['VendorName', 'ModelName', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN',
       'CHMAX', 'PRP'],
      dtype='object')
Train MSE: 0.0002754383374487039
Train MAE: 0.008107756053544803
Train R^2: 0.9999999856724991
Test MSE: 11350.907398928028
Test MAE: 51.15115314288937
Test R^2: 0.7770259465853047
CV MSE: [23516.66084678   870.18422262  4455.59568456  3101.84964874
   980.69658404]
CV MAE: [65.99669142 20.95386589 42.59249071 33.33729342 18.63794896]
CV R^2: [0.49601108 0.81096198 0.61446805 0.84407333 0.88001064]


### 3. (2) 단계에서 얻은 Pipeline 기반 GridSearchCV의 best_estimator모델의 테스트 데이터에 대해 예측 성능 R^2가 85% 이상이 나오도록 모델을 개선하시오.

In [24]:
# 데이터 로드 
data = pd.read_csv("machine_data.csv")
print(data.columns)

# 범주형 변수 더미 변수화
data = pd.get_dummies(data, columns=['VendorName', 'ModelName'])

# 상관계수 행렬 계산
corr_matrix = data.corr()

# PRP 변수와의 상관계수 추출
corr_with_prp = corr_matrix['PRP']

# 절대값 상관계수로 변환 후 정렬
corr_with_prp_abs = np.abs(corr_with_prp)
top_features = corr_with_prp_abs.sort_values(ascending=False).index[1:6]
print("상관도가 높은 특징 5개 :", top_features)

# 선택한 상관도 높은 특징 5개 사용
X = data[top_features]
y = data['PRP']

# 전처리 파이프라인 설정
num_features = X.columns
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features)
])

# 개별 회귀 모델 정의
models = [
    ('lr', LinearRegression()),
    ('sgd', SGDRegressor()),
    ('ridge', Ridge())
]

# 앙상블 모델 정의
ensemble = VotingRegressor(estimators=models)

# 모델 파이프라인 생성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ensemble', ensemble)
])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 서치 설정
params_grid = {
    "ensemble__sgd__alpha": [0.0001, 0.1, 1.0],
    "ensemble__sgd__max_iter": [1000, 2000, 3000],
    "ensemble__ridge__alpha": [0.1, 1.0, 10.0]
}

# GridSearchCV 설정 및 모델 학습
grid_search = GridSearchCV(model_pipeline, params_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

# 최적 모델 및 결과 출력
best_model = grid_search.best_estimator_

# 학습 데이터에 대한 성능 평가
y_train_pred = best_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# 교차 검증을 통한 평균 MSE 계산
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse_mean = np.mean(np.sqrt(-cv_scores))
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
cv_mae_mean = np.mean(np.sqrt(-cv_scores))
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse_mean = np.mean(np.sqrt(-cv_scores))

# 테스트 데이터에 대한 성능 평가
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# 결과 출력
print("Train MSE:", train_mse)
print("Train MAE:", train_mae)
print("Train R^2:", train_r2)
print("CV Mean RMSE:", cv_mse_mean)
print("Test MSE:", test_mse)
print("Test MAE:", test_mae)
print("Test R^2:", test_r2)


Index(['VendorName', 'ModelName', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN',
       'CHMAX', 'PRP'],
      dtype='object')
상관도가 높은 특징 5개 : Index(['MMAX', 'MMIN', 'CACH', 'CHMIN', 'CHMAX'], dtype='object')
Train MSE: 3368.294908152017
Train MAE: 32.97947945927826
Train R^2: 0.8247911002624319
CV Mean RMSE: 65.46718969750248
Test MSE: 7298.193353416181
Test MAE: 48.06469524305879
Test R^2: 0.8566363289362156


### 4. I. 3차 프로그래밍 과제의 기본모델의 (2) 단계와 II. 4차 프로그래밍 과제의 제안 모델의 (2) 단계에서 얻은 학습 데이터와 테스트 데이터에 대한 성능 지표(MSE, MAE, R^2, 교차 검증)를 데이터프레임에 저장하여 이 성능 결과를 바 차트로 시각화하여 학습 데이터와 테스트 데이터에 대한 각 모델의 성능 차이를 비교 분석하시오.

In [43]:
# 4차 프로그래밍 과제 (4)번 문제를 위한 코드

data = {'Model':['Best_estimator', 'Best_estimator', 'Best_estimator'],
       'Data':['Train', 'Test', 'Cross Validation'],
        'MSE':[train_mse, test_mse, -cv_scores_mse],
        'MAE':[train_mae, test_mae, -cv_scores_mae],
        'R^2':[train_r2, test_r2, cv_scores_r2]
       }

result = pd.DataFrame(data)
result.to_csv("pipeline_cpu_df.csv")

In [63]:
df_linear = pd.read_csv("linear_cpu_df.csv")
df_pipeline = pd.read_csv("pipeline_cpu_df.csv")

# 데이터 프레임 생성
data = [df_linear.loc[0], df_linear.loc[1], df_pipeline.loc[0], df_pipeline.loc[1], df_linear.loc[2], df_pipeline.loc[2]]
result = pd.DataFrame(data)
result

,Unnamed: 0,Model,Data,MSE,MAE,R^2
0,0,Linear Regression,Train,1491.03436,24.997057,0.922441
1,1,Linear Regression,Test,9536.323773,45.748933,0.812671
0,0,Best_estimator,Train,0.0002754383374487039,0.008107756053544803,0.9999999856724991
1,1,Best_estimator,Test,11350.907398928028,51.15115314288937,0.7770259465853047
2,2,Linear Regression,Cross Validation,4518.315033,30.389762,-0.805266
2,2,Best_estimator,Cross Validation,[23516.66084678 870.18422262 4455.59568456 ...,[65.99669142 20.95386589 42.59249071 33.337293...,[0.49601108 0.81096198 0.61446805 0.84407333 0...
